In [1]:
!pip install -q transformers datasets evaluate accelerate


In [2]:
!pip install -q git+https://github.com/csebuetnlp/normalizer.git

  Preparing metadata (setup.py) ... done


In [3]:


import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm.auto import tqdm

In [4]:
df = pd.read_csv("/content/Social Media Engagement Dataset.csv")
df.head()

,post_id,timestamp,day_of_week,platform,user_id,location,language,text_content,translated_text_content,hashtags,...,comments_count,impressions,engagement_rate,brand_name,product_name,campaign_name,campaign_phase,user_past_sentiment_avg,user_engagement_growth,buzz_change_rate
0,kcqbs6hxybia,2024-12-09 11:26:15,Monday,Instagram,user_52nwb0a6,"Melbourne, Australia",pt,Just tried the Chromebook from Google. Best pu...,গুগল থেকে Chromebook ব্যবহার করে দেখুন। সর্বকা...,#Food,...,701,18991,0.19319,Google,Chromebook,BlackFriday,Launch,0.0953,-0.3672,19.1
1,vkmervg4ioos,2024-07-28 19:59:26,Sunday,Twitter,user_ucryct98,"Tokyo, Japan",ru,Just saw an ad for Microsoft Surface Laptop du...,স্প্রিংব্লাস্ট ২০২৫-এর সময় মাইক্রোসফট সারফেস ...,"#MustHave, #Food",...,359,52764,0.05086,Microsoft,Surface Laptop,PowerRelease,Post-Launch,0.1369,-0.4510,-42.6
2,memhx4o1x6yu,2024-11-23 14:00:12,Saturday,Reddit,user_7rrev126,"Beijing, China",ru,What's your opinion about Nike's Epic React? ...,নাইকির এপিক রিঅ্যাক্ট সম্পর্কে আপনার মতামত কী?...,"#Promo, #Food, #Trending",...,643,8887,0.45425,Nike,Epic React,BlackFriday,Post-Launch,0.2855,-0.4112,17.4
3,bhyo6piijqt9,2024-09-16 4:35:25,Monday,YouTube,user_4mxuq0ax,"Lagos, Nigeria",en,Bummed out with my new Diet Pepsi from Pepsi! ...,পেপসির নতুন ডায়েট পেপসি দেখে হতাশ! মান নিয়ে ...,"#Reviews, #Sustainable",...,743,6696,0.42293,Pepsi,Diet Pepsi,LaunchWave,Launch,-0.2094,-0.0167,-5.5
4,c9dkiomowakt,2024-09-05 21:03:01,Thursday,Twitter,user_l1vpox2k,"Berlin, Germany",hi,Just tried the Corolla from Toyota. Absolutely...,টয়োটার করোলাটা ট্রাই করলাম। সত্যিই খুব ভালো ল...,"#Health, #Travel",...,703,47315,0.08773,Toyota,Corolla,LocalTouchpoints,Launch,0.6867,0.0807,38.8


In [5]:
df.shape

(12000, 29)

In [6]:
selected_columns = ['text_content', 'translated_text_content', 'sentiment_label']
df = df[selected_columns]
df.head()

,text_content,translated_text_content,sentiment_label
0,Just tried the Chromebook from Google. Best pu...,গুগল থেকে Chromebook ব্যবহার করে দেখুন। সর্বকা...,Positive
1,Just saw an ad for Microsoft Surface Laptop du...,স্প্রিংব্লাস্ট ২০২৫-এর সময় মাইক্রোসফট সারফেস ...,Negative
2,What's your opinion about Nike's Epic React? ...,নাইকির এপিক রিঅ্যাক্ট সম্পর্কে আপনার মতামত কী?...,Negative
3,Bummed out with my new Diet Pepsi from Pepsi! ...,পেপসির নতুন ডায়েট পেপসি দেখে হতাশ! মান নিয়ে ...,Negative
4,Just tried the Corolla from Toyota. Absolutely...,টয়োটার করোলাটা ট্রাই করলাম। সত্যিই খুব ভালো ল...,Positive


In [7]:
print(df['sentiment_label'].value_counts())

sentiment_label
Negative    4854
Positive    4839
Neutral     2307
Name: count, dtype: int64


In [8]:
df.isnull().values.any()

np.False_

In [9]:
df.isnull().sum()

,0
text_content,0
translated_text_content,0
sentiment_label,0


In [10]:
def light_preprocess(text):
    if pd.isna(text):
        return ""
    text = str(text).strip()
    # Only collapse multiple spaces, keep everything else
    text = re.sub(r'\s+', ' ', text)
    return text

df['clean_text'] = df['translated_text_content'].apply(light_preprocess)

In [12]:
import unicodedata

df['clean_text'] = df['translated_text_content'].apply(light_preprocess)
pd.set_option('display.max_colwidth', None)
print(df[['translated_text_content', 'clean_text']].head().to_string(index=False))

                                                                                              translated_text_content                                                                                                            clean_text
             গুগল থেকে Chromebook ব্যবহার করে দেখুন। সর্বকালের সেরা কেনাকাটা। #খাবার আপনার মতামত শুনতে সত্যিই আগ্রহী!              গুগল থেকে Chromebook ব্যবহার করে দেখুন। সর্বকালের সেরা কেনাকাটা। #খাবার আপনার মতামত শুনতে সত্যিই আগ্রহী!
স্প্রিংব্লাস্ট ২০২৫-এর সময় মাইক্রোসফট সারফেস ল্যাপটপের একটা বিজ্ঞাপন দেখলাম। টাকার মূল্য নেই। #অবশ্যই খাওয়া, #খাবার। স্প্রিংব্লাস্ট ২০২৫-এর সময় মাইক্রোসফট সারফেস ল্যাপটপের একটা বিজ্ঞাপন দেখলাম। টাকার মূল্য নেই। #অবশ্যই খাওয়া, #খাবার।
          নাইকির এপিক রিঅ্যাক্ট সম্পর্কে আপনার মতামত কী? #প্রচার, #খাবার, #ট্রেন্ডিং আপনার মতামত শুনতে সত্যিই আগ্রহী!           নাইকির এপিক রিঅ্যাক্ট সম্পর্কে আপনার মতামত কী? #প্রচার, #খাবার, #ট্রেন্ডিং আপনার মতামত শুনতে সত্যিই আগ্রহী!
                                              পেপসির নতু

In [13]:
df.head(3)

,text_content,translated_text_content,sentiment_label,clean_text
0,Just tried the Chromebook from Google. Best purchase ever. #Food Really interested in hearing your thoughts!,গুগল থেকে Chromebook ব্যবহার করে দেখুন। সর্বকালের সেরা কেনাকাটা। #খাবার আপনার মতামত শুনতে সত্যিই আগ্রহী!,Positive,গুগল থেকে Chromebook ব্যবহার করে দেখুন। সর্বকালের সেরা কেনাকাটা। #খাবার আপনার মতামত শুনতে সত্যিই আগ্রহী!
1,"Just saw an ad for Microsoft Surface Laptop during the SpringBlast2025. Not worth the money. #MustHave, #Food","স্প্রিংব্লাস্ট ২০২৫-এর সময় মাইক্রোসফট সারফেস ল্যাপটপের একটা বিজ্ঞাপন দেখলাম। টাকার মূল্য নেই। #অবশ্যই খাওয়া, #খাবার।",Negative,"স্প্রিংব্লাস্ট ২০২৫-এর সময় মাইক্রোসফট সারফেস ল্যাপটপের একটা বিজ্ঞাপন দেখলাম। টাকার মূল্য নেই। #অবশ্যই খাওয়া, #খাবার।"
2,"What's your opinion about Nike's Epic React? #Promo, #Food, #Trending Really interested in hearing your thoughts!","নাইকির এপিক রিঅ্যাক্ট সম্পর্কে আপনার মতামত কী? #প্রচার, #খাবার, #ট্রেন্ডিং আপনার মতামত শুনতে সত্যিই আগ্রহী!",Negative,"নাইকির এপিক রিঅ্যাক্ট সম্পর্কে আপনার মতামত কী? #প্রচার, #খাবার, #ট্রেন্ডিং আপনার মতামত শুনতে সত্যিই আগ্রহী!"


In [14]:
from sklearn.model_selection import train_test_split


text_column = 'clean_text'          # ← change to your actual raw-ish column if possible
label_column = 'sentiment_label'

X = df[text_column]
y = df[label_column]

# First split: train + (val+test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.25,               # 75% train, 25% → val+test
    random_state=42,
    stratify=y
)

# Second split: val / test (50/50 of the 25%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,                # → ~12.5% val, ~12.5% test
    random_state=42,
    stratify=y_temp
)

print(f"Train: {len(X_train):,} rows ({len(X_train)/len(df):.1%})")
print(f"Val:   {len(X_val):,} rows   ({len(X_val)/len(df):.1%})")
print(f"Test:  {len(X_test):,} rows  ({len(X_test)/len(df):.1%})")

# Optional: quick class balance check
print("\nClass distribution in train:")
print(y_train.value_counts(normalize=True).round(3))

Train: 9,000 rows (75.0%)
Val:   1,500 rows   (12.5%)
Test:  1,500 rows  (12.5%)

Class distribution in train:
sentiment_label
Negative    0.405
Positive    0.403
Neutral     0.192
Name: proportion, dtype: float64


In [15]:
label2id = {
    'Positive': 0,
    'Neutral':  1,
    'Negative': 2
}

y_train = y_train.map(label2id)
y_val   = y_val.map(label2id)
y_test  = y_test.map(label2id)

print("Train labels sample:", y_train.head().tolist())
print("Unique train labels:", y_train.unique())           # should show [0 1 2]
print("Any NaN left?", y_train.isna().any())              # should be False

Train labels sample: [0, 1, 0, 0, 0]
Unique train labels: [0 1 2]
Any NaN left? False


In [16]:


from normalizer import normalize

# ────────────────────────────────────────────────
#  Improved SentimentDataset – with normalization & clean return
# ────────────────────────────────────────────────

MAX_LEN = 160   # good starting point for product/social media reviews

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=MAX_LEN):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):  # better to use idx instead of item
        text = str(self.texts[idx])

        # ─── REQUIRED for banglabert ───
        text = normalize(text)

        label = self.labels[idx]

        # Safety check: skip if label is NaN (should be fixed earlier)
        if pd.isna(label):
            # You can return a dummy or skip – but better to fix labels before
            label = 0  # fallback – but ideally fix mapping

        # Corrected: Call the tokenizer directly instead of using .encode_plus()
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            # Removed 'text_content' – not needed for training
            'input_ids':      encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels':         torch.tensor(int(label), dtype=torch.long)
        }

# ────────────────────────────────────────────────
#  Load tokenizer
# ────────────────────────────────────────────────

MODEL_NAME = 'csebuetnlp/banglabert'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print("Tokenizer loaded:", MODEL_NAME)

# ────────────────────────────────────────────────
#  Create datasets – run this AFTER fixing label mapping
# ────────────────────────────────────────────────

train_dataset = SentimentDataset(
    texts=X_train.values,
    labels=y_train.values,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

val_dataset = SentimentDataset(
    texts=X_val.values,
    labels=y_val.values,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

test_dataset = SentimentDataset(
    texts=X_test.values,
    labels=y_test.values,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

# Quick validation
print("\nDataset sizes:", len(train_dataset), len(val_dataset), len(test_dataset))

# Very important check – make sure labels are integers now
sample = train_dataset[0]
print("Sample label type:", sample['labels'].dtype)          # should be torch.int64
print("Sample label value:", sample['labels'].item())        # should be 0,1 or 2
print("input_ids length:", len(sample['input_ids']))         # should be 160

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Tokenizer loaded: csebuetnlp/banglabert

Dataset sizes: 9000 1500 1500
Sample label type: torch.int64
Sample label value: 0
input_ids length: 160


In [20]:
!pip install -q git+https://github.com/csebuetnlp/normalizer.git

  Preparing metadata (setup.py) ... done


In [23]:
# ────────────────────────────────────────────────
#  Create DataLoaders + device setup
# ────────────────────────────────────────────────

import torch
from torch.utils.data import DataLoader

# Recommended batch size for T4 GPU (Colab free tier)
# Reduce to 8 or 12 if you get CUDA out of memory later
BATCH_SIZE = 16
NUM_WORKERS = 2   # increase to 4 if your CPU allows

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True if torch.cuda.is_available() else False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True if torch.cuda.is_available() else False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"\nNumber of batches:")
print(f"  Train: {len(train_loader):>4}")
print(f"  Val:   {len(val_loader):>4}")
print(f"  Test:  {len(test_loader):>4}")

# Quick batch shape check (run once)
batch = next(iter(train_loader))
print("\nBatch shapes:")
print("  input_ids:     ", batch['input_ids'].shape)      # expected: [16, 160]
print("  attention_mask:", batch['attention_mask'].shape)  # expected: [16, 160]
print("  labels:        ", batch['labels'].shape)         # expected: [16]
print("  labels dtype:  ", batch['labels'].dtype)         # expected: torch.int64

Using device: cuda
GPU available: True
GPU name: Tesla T4

Number of batches:
  Train:  563
  Val:     94
  Test:    94

Batch shapes:
  input_ids:      torch.Size([16, 160])
  attention_mask: torch.Size([16, 160])
  labels:         torch.Size([16])
  labels dtype:   torch.int64


In [24]:
# ────────────────────────────────────────────────
#  Create DataLoaders + device & quick check
# ────────────────────────────────────────────────

import torch
from torch.utils.data import DataLoader

# Settings – adjust if needed
BATCH_SIZE = 16          # ↓ to 8 or 12 if you get "CUDA out of memory"
NUM_WORKERS = 2          # ↑ to 4 if your CPU is strong

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Current GPU memory used: {torch.cuda.memory_allocated(0)/1e9:.2f} GB")

# ─── DataLoaders ───
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=torch.cuda.is_available(),
    drop_last=True              # avoids uneven last batch issues
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=torch.cuda.is_available()
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=torch.cuda.is_available()
)

print("\nBatches created:")
print(f"  Train: {len(train_loader):>6} batches")
print(f"  Val:   {len(val_loader):>6} batches")
print(f"  Test:  {len(test_loader):>6} batches")

# Quick sanity check on first batch
try:
    batch = next(iter(train_loader))
    print("\nFirst batch shapes:")
    print(f"  input_ids:      {batch['input_ids'].shape}")
    print(f"  attention_mask: {batch['attention_mask'].shape}")
    print(f"  labels:         {batch['labels'].shape}")
    print(f"  labels dtype:   {batch['labels'].dtype}")
    print(f"  labels sample:  {batch['labels'][:10].tolist()}")
except Exception as e:
    print("Error loading batch:", str(e))

Device: cuda
GPU: Tesla T4
Current GPU memory used: 0.00 GB

Batches created:
  Train:    562 batches
  Val:       94 batches
  Test:      94 batches

First batch shapes:
  input_ids:      torch.Size([16, 160])
  attention_mask: torch.Size([16, 160])
  labels:         torch.Size([16])
  labels dtype:   torch.int64
  labels sample:  [1, 1, 2, 2, 2, 2, 2, 0, 2, 2]


In [25]:
# ────────────────────────────────────────────────
#  BanglaBERT + BiLSTM Model
# ────────────────────────────────────────────────

import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel

class BanglaBERTBiLSTM(nn.Module):
    def __init__(self,
                 num_classes=3,           # Positive=0, Neutral=1, Negative=2
                 lstm_hidden_size=256,
                 lstm_num_layers=1,
                 dropout=0.3):
        super().__init__()

        # Load pretrained BanglaBERT
        self.bert = AutoModel.from_pretrained(MODEL_NAME)
        self.bert_dim = self.bert.config.hidden_size  # usually 768

        # Optional: freeze early layers to stabilize training & save memory
        # Uncomment if you want to freeze first 8 layers (common for small datasets)
        # for name, param in self.bert.named_parameters():
        #     if "encoder.layer" in name and int(name.split(".")[3]) < 8:
        #         param.requires_grad = False

        # BiLSTM on top of BERT hidden states
        self.bilstm = nn.LSTM(
            input_size=self.bert_dim,
            hidden_size=lstm_hidden_size,
            num_layers=lstm_num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if lstm_num_layers > 1 else 0.0
        )

        # Dropout + final classifier
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(lstm_hidden_size * 2, num_classes)  # ×2 because bidirectional

    def forward(self, input_ids, attention_mask, labels=None):
        # Get BERT outputs
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # Take last hidden states (batch, seq_len, hidden_dim)
        sequence_output = outputs.last_hidden_state

        # Pass through BiLSTM
        lstm_out, _ = self.bilstm(sequence_output)  # (batch, seq_len, hidden*2)

        # Global mean pooling – very stable & effective for classification
        pooled_output = torch.mean(lstm_out, dim=1)  # (batch, hidden*2)

        # Alternative: use [CLS] token instead (uncomment if you prefer)
        # pooled_output = sequence_output[:, 0, :]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        if labels is not None:
            loss = F.cross_entropy(logits, labels)
            return loss, logits
        return logits

# ─── Instantiate model ───
model = BanglaBERTBiLSTM(num_classes=3).to(device)

print("Model created and moved to:", device)
print(f"Total trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

ElectraModel LOAD REPORT from: csebuetnlp/banglabert
Key                                               | Status     |  | 
--------------------------------------------------+------------+--+-
discriminator_predictions.dense_prediction.weight | UNEXPECTED |  | 
discriminator_predictions.dense_prediction.bias   | UNEXPECTED |  | 
electra.embeddings.position_ids                   | UNEXPECTED |  | 
discriminator_predictions.dense.weight            | UNEXPECTED |  | 
discriminator_predictions.dense.bias              | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Model created and moved to: cuda
Total trainable parameters: 112,129,539


In [26]:
# ────────────────────────────────────────────────
#  Optimizer, Scheduler & Training Loop
# ────────────────────────────────────────────────

from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, classification_report
import time

# ─── Hyperparameters ───
NUM_EPOCHS = 4              # start with 3–5; monitor val loss
LEARNING_RATE = 2e-5        # typical for BERT fine-tuning
WARMUP_RATIO = 0.1          # 10% of total steps

# Calculate total steps
total_steps = len(train_loader) * NUM_EPOCHS
warmup_steps = int(total_steps * WARMUP_RATIO)

# ─── Optimizer & Scheduler ───
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# ─── Training Loop ───
best_val_acc = 0.0
best_epoch = 0

print(f"Starting training for {NUM_EPOCHS} epochs...")
print(f"Total steps: {total_steps:,} | Warmup: {warmup_steps:,}")

for epoch in range(NUM_EPOCHS):
    start_time = time.time()

    # ─── Train ───
    model.train()
    total_train_loss = 0

    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        loss, logits = model(input_ids, attention_mask, labels)
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # prevent exploding gradients
        optimizer.step()
        scheduler.step()

        if step % 50 == 0:
            print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Step {step}/{len(train_loader)} | Loss: {loss.item():.4f}")

    avg_train_loss = total_train_loss / len(train_loader)

    # ─── Validation ───
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            loss, logits = model(input_ids, attention_mask, labels)
            total_val_loss += loss.item()

            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    val_acc = accuracy_score(all_labels, all_preds)

    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS} completed in {time.time() - start_time:.1f}s")
    print(f"  Train Loss: {avg_train_loss:.4f}")
    print(f"  Val   Loss: {avg_val_loss:.4f}")
    print(f"  Val Accuracy: {val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch + 1
        torch.save(model.state_dict(), "best_banglabert_bilstm.pt")
        print(f"  → New best model saved! Val Acc: {val_acc:.4f}")

    print("-" * 60)

print(f"\nTraining finished.")
print(f"Best validation accuracy: {best_val_acc:.4f} at epoch {best_epoch}")

Starting training for 4 epochs...
Total steps: 2,248 | Warmup: 224
Epoch 1/4 | Step 0/562 | Loss: 1.1324
Epoch 1/4 | Step 50/562 | Loss: 1.0737
Epoch 1/4 | Step 100/562 | Loss: 1.0935
Epoch 1/4 | Step 150/562 | Loss: 0.6190
Epoch 1/4 | Step 200/562 | Loss: 0.3409
Epoch 1/4 | Step 250/562 | Loss: 0.1160
Epoch 1/4 | Step 300/562 | Loss: 0.2348
Epoch 1/4 | Step 350/562 | Loss: 0.1932
Epoch 1/4 | Step 400/562 | Loss: 0.0650
Epoch 1/4 | Step 450/562 | Loss: 0.1470
Epoch 1/4 | Step 500/562 | Loss: 0.0062
Epoch 1/4 | Step 550/562 | Loss: 0.0053

Epoch 1/4 completed in 297.7s
  Train Loss: 0.3907
  Val   Loss: 0.1326
  Val Accuracy: 0.9347
  → New best model saved! Val Acc: 0.9347
------------------------------------------------------------
Epoch 2/4 | Step 0/562 | Loss: 0.0607
Epoch 2/4 | Step 50/562 | Loss: 0.1123
Epoch 2/4 | Step 100/562 | Loss: 0.3050
Epoch 2/4 | Step 150/562 | Loss: 0.1104
Epoch 2/4 | Step 200/562 | Loss: 0.2119
Epoch 2/4 | Step 250/562 | Loss: 0.0028
Epoch 2/4 | Step 300

In [28]:
# Load best model
model.load_state_dict(torch.load("best_banglabert_bilstm.pt"))
model.eval()

all_test_preds = []
all_test_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Corrected: Only expect one return value (logits) when labels are not passed
        logits = model(input_ids, attention_mask)
        preds = torch.argmax(logits, dim=1).cpu().numpy()

        all_test_preds.extend(preds)
        all_test_labels.extend(labels.cpu().numpy())

print("Test Accuracy:", accuracy_score(all_test_labels, all_test_preds))
print("\nClassification Report:")
print(classification_report(all_test_labels, all_test_preds, target_names=['Positive', 'Neutral', 'Negative']))

Test Accuracy: 0.9393333333333334

Classification Report:
              precision    recall  f1-score   support

    Positive       0.87      1.00      0.93       605
     Neutral       1.00      0.90      0.94       288
    Negative       0.99      0.90      0.95       607

    accuracy                           0.94      1500
   macro avg       0.95      0.93      0.94      1500
weighted avg       0.95      0.94      0.94      1500



In [31]:
from normalizer import normalize

# Function to predict sentiment for a single sentence
def predict_sentiment(text, model, tokenizer, device, max_len=MAX_LEN):
    # Apply normalization (same as during training)
    text = normalize(text)

    # Corrected: Call the tokenizer directly instead of using .encode_plus()
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        probabilities = F.softmax(outputs, dim=1)

    # Map numerical labels back to string labels
    id2label = {v: k for k, v in label2id.items()}

    predicted_class_id = torch.argmax(probabilities, dim=1).item()
    predicted_sentiment = id2label[predicted_class_id]

    # Format probabilities
    probs_dict = {
        id2label[0]: probabilities[0][0].item(),
        id2label[1]: probabilities[0][1].item(),
        id2label[2]: probabilities[0][2].item()
    }

    return {
        'text': text,
        'predicted': predicted_sentiment,
        'probabilities': probs_dict
    }

# Some real-life style Bangla product review examples
test_sentences = [
    "এই ফোনটা দেখতে অনেক সুন্দর, কিন্তু ব্যাটারি খুব দ্রুত শেষ হয়ে যায় 😞",
    "আমার অর্ডারটা সময়মতো ডেলিভারি হয়েছে, প্রোডাক্ট কোয়ালিটি দারুণ!",
    "দাম অনেক বেশি, কোনো ভ্যালু ফর মানি নাই।",
    "খুবই ভালো লাগলো, ১০/১০ রেকমেন্ড করছি ❤️",
    "প্যাকেজিং খারাপ ছিল, প্রোডাক্ট একটু ভাঙা এসেছে।",
    "এতো সুন্দর ডিজাইন দেখে অবাক হয়ে গেছি!",
    "সার্ভিস খুব খারাপ, টাকা ফেরত চাই।",
    "ঠিকঠাক আছে, কোনো সমস্যা নাই।"
]

print("Prediction Results:\n" + "="*70)

for sentence in test_sentences:
    result = predict_sentiment(sentence, model, tokenizer, device)
    print(f"Text: {result['text']}")
    print(f"→ Predicted: {result['predicted']}")
    print(f"  Probabilities: Positive {result['probabilities']['Positive']:.2%} | "
          f"Neutral {result['probabilities']['Neutral']:.2%} | "
          f"Negative {result['probabilities']['Negative']:.2%}")
    print("-"*70)

Prediction Results:
Text: এই ফোনটা দেখতে অনেক সুন্দর, কিন্তু ব্যাটারি খুব দ্রুত শেষ হয়ে যায় 😞
→ Predicted: Negative
  Probabilities: Positive 0.03% | Neutral 0.03% | Negative 99.94%
----------------------------------------------------------------------
Text: আমার অর্ডারটা সময়মতো ডেলিভারি হয়েছে, প্রোডাক্ট কোয়ালিটি দারুণ!
→ Predicted: Positive
  Probabilities: Positive 95.15% | Neutral 4.46% | Negative 0.39%
----------------------------------------------------------------------
Text: দাম অনেক বেশি, কোনো ভ্যালু ফর মানি নাই।
→ Predicted: Negative
  Probabilities: Positive 0.02% | Neutral 0.03% | Negative 99.95%
----------------------------------------------------------------------
Text: খুবই ভালো লাগলো, ১০/১০ রেকমেন্ড করছি ❤️
→ Predicted: Positive
  Probabilities: Positive 99.89% | Neutral 0.08% | Negative 0.03%
----------------------------------------------------------------------
Text: প্যাকেজিং খারাপ ছিল, প্রোডাক্ট একটু ভাঙা এসেছে।
→ Predicted: Negative
  Probabilities: Positive 0.